In [2]:
import os
import cv2
import numpy as np
import torch
import tensorflow as tf
from yolov6.utils.nms import non_max_suppression
from tqdm import tqdm
import time

import torch.nn.functional as F
mean, std = [83.27841169 ,90.18828133, 93.20692581], [58.87947213, 59.70446263, 59.42014208]

color_map = {
    0: [0, 0, 0],  # 背景
    1: [255, 0, 0],  # 眉毛
    2: [0, 255, 0],  # 眼睛
    3: [0, 0, 255],  # 鼻子
    4: [255, 255, 0],  # 嘴巴
    5: [255, 0, 255]  # 脸部
}

def box_convert(x):
        # Convert boxes with shape [n, 4] from [x1, y1, x2, y2] to [x, y, w, h] where x1y1=top-left, x2y2=bottom-right
        y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
        y[:, 0] = (x[:, 0] + x[:, 2]) / 2  # x center
        y[:, 1] = (x[:, 1] + x[:, 3]) / 2  # y center
        y[:, 2] = x[:, 2] - x[:, 0]  # width
        y[:, 3] = x[:, 3] - x[:, 1]  # height
        return y
    
def rescale(ori_shape, boxes, target_shape):
    '''Rescale the output to the original image shape'''
    ratio = min(ori_shape[0] / target_shape[0], ori_shape[1] / target_shape[1])
    padding = (ori_shape[1] - target_shape[1] * ratio) / 2, (ori_shape[0] - target_shape[0] * ratio) / 2

    boxes[:, [0, 2]] -= padding[0]
    boxes[:, [1, 3]] -= padding[1]
    boxes[:, :4] /= ratio

    boxes[:, 0].clamp_(0, target_shape[1])  # x1
    boxes[:, 1].clamp_(0, target_shape[0])  # y1
    boxes[:, 2].clamp_(0, target_shape[1])  # x2
    boxes[:, 3].clamp_(0, target_shape[0])  # y2

    return boxes

def generate_colors(i, bgr=False):
    hex = ('FF3838', 'FF9D97', 'FF701F', 'FFB21D', 'CFD231', '48F90A', '92CC17', '3DDB86', '1A9334', '00D4BB',
           '2C99A8', '00C2FF', '344593', '6473FF', '0018EC', '8438FF', '520085', 'CB38FF', 'FF95C8', 'FF37C7')
    palette = []
    for iter in hex:
        h = '#' + iter
        palette.append(tuple(int(h[1 + i:1 + i + 2], 16) for i in (0, 2, 4)))
    num = len(palette)
    color = palette[int(i) % num]
    return (color[2], color[1], color[0]) if bgr else color

def plot_box_and_label(image, lw, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255), font=cv2.FONT_HERSHEY_COMPLEX):
    # Add one xyxy box to image with label
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
    if label:
        tf = max(lw - 1, 1)  # font thickness
        w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(image, label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2), font, lw / 3, txt_color,
                    thickness=tf, lineType=cv2.LINE_AA)
        
def segmap_to_color(segmap):
    color_image = np.zeros((segmap.shape[0], segmap.shape[1 ], 3), dtype=np.uint8)
    for i in range(6):
        color_image[segmap == i] = color_map[i]
    return color_image

def colorize_image(image, segmap):
    color_segmap = segmap_to_color(segmap)
    result = cv2.addWeighted(image, 0.7, color_segmap, 0.3, 20.0)
    return result

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_image(image, input_shape, uint8=False):
    image = cv2.resize(image, input_shape)
    if not uint8:
        image = image.astype(np.float32)
        image /= 255.0
    else:
        image = image.astype(np.float32) #/ 255.0
#         image = (image - mean) / std
        image = (image ).astype(np.int8)
    image = image[:,:,::-1]
    
    image = np.expand_dims(image, axis=0).transpose((0,3,1,2))
    return image


def postprocess_segmentation(segmentation, image):
#     import pdb
#     pdb.set_trace()
    # print("seg & img shape :")
    # print(segmentation.shape)
    # print(image.shape[:2])
    segmentation = F.interpolate(torch.Tensor(segmentation), size=(image.shape[:2]), mode='bilinear', align_corners=True)
#     segmentation=segmentation[0].permute(1,2,0).numpy()


    
    segmentation = torch.argmax(segmentation, 1).squeeze()
    segmentation = segmentation.numpy().astype(np.uint8)
    return segmentation, colorize_image(image, segmentation)
    

def process_images(input_folder, model, input_shape=(576, 960), output_folder="output",hide_labels=False, hide_conf=False):
    input_details = model.get_input_details()
    output_details = model.get_output_details()

    os.makedirs(output_folder, exist_ok=True)
    
    
    t1,t2,t3,t4 = 0,0,0,0
    dx = os.listdir(input_folder)
    dx = sorted(dx)[:10]
    # print("dx : 0" , dx)
    len1 = len(dx)
    with open("submission.csv", 'w') as fpx:
        fpx.write('image_filename,label_id,x,y,w,h,confidence\n')
        
        for image_name in tqdm(dx):
            start = time.time()
            image_path = os.path.join(input_folder, image_name)
            image_name = os.path.basename(image_path)
            image = cv2.imread(image_path)
            h,w,c = image.shape

            # Preprocess the image
            preprocessed_image = preprocess_image(image, input_shape, uint8=False)
            # print(preprocessed_image.shape)

            # Set the input tensor
            model.set_tensor(input_details[0]['index'], preprocessed_image)

            # Invoke the model
            t1 += (time.time()-start)
            start = time.time()
            
            model.invoke()
            
            t2 += (time.time()-start)
            start = time.time()
            
            # Retrieve the output tensors (segmentation and detection)
    #         print(model.get_tensor(output_details[0]['index']).shape)
            detections = model.get_tensor(output_details[1]['index']) # 0
            segmentation = model.get_tensor(output_details[0]['index']) # 1
            # print("-----shape-----")
            # print(detections.shape)
            # print(segmentation.shape)
#             import pdb
#             pdb.set_trace()

            # print("#"*80)
            # print("detections : ")
            # print(type(detections))
            # print(detections.shape)
            # print(detections)
            # print("#"*80)
            # assert 1 == 2

            # Postprocess the output tensors
            segmentation_map, seg_color = postprocess_segmentation(segmentation, image)

            classes:Optional[List[int]] = None
            det = non_max_suppression(torch.from_numpy(detections), 0.2,0.7, classes, False, max_det=1000)[0]
            print("det :" , det)

            t3 += (time.time()-start)
            start = time.time()

            segmentation_output_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_segmap_color.jpg")
            cv2.imwrite(segmentation_output_path, seg_color)

            segmentation_output_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_segmap.png")
            cv2.imwrite(segmentation_output_path, segmentation_map)
    #         print(preprocessed_image.shape)
            gn = torch.tensor(image.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            txt_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_det.txt")
            if len(det):
                # print(det)
#                 import pdb
#                 pdb.set_trace()
                det[:, :4] = rescale(preprocessed_image.shape[1:3], det[:, :4], image.shape).round()
#                 print(preprocessed_image.shape[1:3])
                for *xyxy, conf, cls in reversed(det):
                    class_num = int(cls)
    #                 print(xyxy)
                    xyxy = np.array(xyxy).astype(np.int16)
                    label = None if hide_labels else (class_num if hide_conf else f'{class_num} {conf:.2f}')
                    plot_box_and_label(image, max(round(sum(image.shape) / 2 * 0.003), 2), xyxy, 
                                       label, color=generate_colors(4, True))

                image_width, image_height = image.shape[1], image.shape[0]
                

                for *xyxy, conf, cls in reversed(det):
                    xywh = (box_convert(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    line = (cls, *xywh, conf)
    #                 with open(txt_path, 'a') as f:
    #                     f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    class_id, x, y, w, h, confidence = map(float, line)
                    x_min = int((x - w/2) * image_width)
                    y_min = int((y - h/2) * image_height)
                    x_max = int((x + w/2) * image_width)
                    y_max = int((y + h/2) * image_height)
                    fpx.write(f"{image_name+'.jpg'},{int(class_id)+1},{x_min},{y_min},{x_max - x_min},{y_max - y_min},{confidence}\n")
                
                detection_output_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_detections_new.jpg")
                cv2.imwrite(detection_output_path , image)

    #         for det in det:
    #             x, y, w, h, confidence, class_id = det
    #             x, y, w, h = int(x * image.shape[1]), int(y * image.shape[0]), int(w * image.shape[1]), int(h * image.shape[0])
    #             cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    #             cv2.putText(image, f"{class_id}: {confidence:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            detection_output_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_detections.jpg")
            cv2.imwrite(detection_output_path, image)
            t4 += (time.time()-start)

        print(f'test time: loading: {t1/len1}, inference: {t2/len1}, nms: {t3/len1}, writing: {t4/len1}')
        
input_folder = "test_inference"
model_path = "0417/saved_model/last_ckpt_opset11_0417_float16.tflite"
output_folder = "test_inference/output/teacher"
model = load_tflite_model(model_path)

process_images(input_folder, model, input_shape=(1280, 1280), output_folder=output_folder) # 640 , 384

ValueError: Didn't find op for builtin opcode 'CONV_2D' version '6'. An older version of this builtin might be supported. Are you using an old TFLite binary with a newer model?
Registration failed.
